# Задача

Реализовать класс `MyBinaryLogisticRegression` для работы с логистической регрессией. Обеспечить возможность использования `l1`, `l2` и `l1l2` регуляризации и реализовать слудующие методы решения оптимизационной задачи:

*   Градиентный спуск
*   Стохастический градиентный спуск
*   Метод Ньютона

Обосновать применимость/не применимость того или иного метода оптимизации в случае использованного типа регуляризации.



In [1]:

import numpy as np
import pandas as pd


class MyBinaryLogisticRegression:
    """
    Простой бинарный классификатор на логистической регрессии.
    Поддерживает разные варианты регуляризации и методы оптимизации.
    """

    def __init__(
        self,
        reg_type: str = "none",
        reg_strength: float = 0.0,
        l1_ratio: float = 0.5,
        lr: float = 0.1,
        max_iter: int = 1000,
        tol: float = 1e-5,
        batch_size: int = 32,
        random_state: int = 42,
    ):
        # Основные гиперпараметры
        self.reg_type = reg_type  # none | l1 | l2 | l1l2
        self.reg_strength = reg_strength  # коэффициент штрафа
        self.l1_ratio = l1_ratio  # доля l1 в смешанной регуляризации
        self.lr = lr  # шаг градиентного спуска
        self.max_iter = max_iter  # максимум итераций
        self.tol = tol  # критерий остановки по норме шага/градиента
        self.batch_size = batch_size  # размер мини-батча для SGD
        self.random_state = random_state  # фиксируем сид для воспроизводимости

        # Атрибуты, которые наполняются после обучения
        self.coefs_ = None
        self.feature_names_in_ = None

        np.random.seed(self.random_state)

    def _sigmoid(self, z: np.array) -> np.array:
        """Численно стабильная сигмоида."""
        z = np.clip(z, -500, 500)
        return 1 / (1 + np.exp(-z))

    def _prepare_X(self, X):
        """Добавляем столбец с единицами для свободного члена."""
        if isinstance(X, pd.DataFrame):
            self.feature_names_in_ = ["bias"] + list(X.columns)
            X_array = X.values.astype(float)
        else:
            X_array = np.asarray(X, dtype=float)
            if self.feature_names_in_ is None:
                self.feature_names_in_ = ["bias"] + [
                    f"x{i}" for i in range(1, X_array.shape[1] + 1)
                ]
        bias = np.ones((X_array.shape[0], 1))
        return np.hstack([bias, X_array])

    def _reg_gradient(self, w: np.array) -> np.array:
        """Градиент по части с регуляризацией."""
        grad = np.zeros_like(w)
        if self.reg_strength == 0 or self.reg_type == "none":
            return grad
        if self.reg_type == "l1":
            grad += self.reg_strength * np.sign(w)
        elif self.reg_type == "l2":
            grad += 2 * self.reg_strength * w
        elif self.reg_type == "l1l2":
            grad += self.reg_strength * (
                self.l1_ratio * np.sign(w) + 2 * (1 - self.l1_ratio) * w
            )
        grad[0] = 0.0  # не штрафуем свободный член
        return grad

    def _reg_hessian(self, size: int) -> np.array:
        """Добавка в гессиан от регуляризации."""
        if self.reg_type == "l2":
            diag = np.zeros(size)
            diag[1:] = 2 * self.reg_strength
            return np.diag(diag)
        if self.reg_type == "l1l2":
            diag = np.zeros(size)
            diag[1:] = 2 * self.reg_strength * (1 - self.l1_ratio)
            return np.diag(diag)
        if self.reg_type == "l1":
            eye = np.eye(size) * 1e-4  # маленькая добавка, чтобы матрица была невырожденной
            eye[0, 0] = 0.0
            return eye
        return np.zeros((size, size))

    def _gradient(self, X: np.array, y: np.array, w: np.array) -> np.array:
        """Полный градиент функции потерь."""
        preds = self._sigmoid(X @ w)
        error = preds - y
        grad = X.T @ error / len(y)
        grad += self._reg_gradient(w)
        return grad

    def _hessian(self, X: np.array, w: np.array) -> np.array:
        """Гессиан для метода Ньютона."""
        preds = self._sigmoid(X @ w)
        weights = preds * (1 - preds)
        weighted_X = X * weights[:, np.newaxis]
        hess = X.T @ weighted_X / len(weights)
        hess += self._reg_hessian(X.shape[1])
        return hess

    def fit(self, X: pd.DataFrame, y: pd.Series, method: str = "gd"):
        """
        Обучение модели.
        method: gd (градиентный спуск) | sgd (стохастический) | newton (метод Ньютона)
        """
        X_mat = self._prepare_X(X)
        y_array = np.asarray(y, dtype=float).reshape(-1)
        self.coefs_ = np.zeros(X_mat.shape[1], dtype=float)

        for iteration in range(self.max_iter):
            if method == "gd":
                grad = self._gradient(X_mat, y_array, self.coefs_)
                self.coefs_ -= self.lr * grad
                if np.linalg.norm(grad) < self.tol:
                    break
            elif method == "sgd":
                indices = np.arange(len(y_array))
                np.random.shuffle(indices)
                for start in range(0, len(indices), self.batch_size):
                    batch_idx = indices[start : start + self.batch_size]
                    grad = self._gradient(
                        X_mat[batch_idx], y_array[batch_idx], self.coefs_
                    )
                    self.coefs_ -= self.lr * grad
                full_grad = self._gradient(X_mat, y_array, self.coefs_)
                if np.linalg.norm(full_grad) < self.tol:
                    break
            elif method == "newton":
                grad = self._gradient(X_mat, y_array, self.coefs_)
                hess = self._hessian(X_mat, self.coefs_)
                try:
                    step = np.linalg.solve(hess, grad)
                except np.linalg.LinAlgError:
                    step = np.linalg.pinv(hess) @ grad
                self.coefs_ -= step
                if np.linalg.norm(step) < self.tol:
                    break
            else:
                raise ValueError("Неизвестный метод оптимизации")
        return self

    def predict_proba(self, X):
        """Вероятности класса 1."""
        X_mat = self._prepare_X(X)
        return self._sigmoid(X_mat @ self.coefs_)

    def predict(self, X):
        """Метки классов по порогу 0.5."""
        probs = self.predict_proba(X)
        return (probs >= 0.5).astype(int)

    def score(self, X, y):
        """Доля правильных ответов (accuracy)."""
        preds = self.predict(X)
        y_array = np.asarray(y)
        return (preds == y_array).mean()


Продемонстрировать применение реализованного класса на датасете про пингвинов (целевая переменная — вид пингвина). Рассмотреть все возможные варианты (регуляризация/оптимизация). Для категориального признака `island` реализовать самостоятельно преобразование `Target Encoder`, сравнить результаты классификации с `one-hot`. В качестве метрики использовать `f1-score`.

In [2]:

# Загружаем данные про пингвинов и смотрим, что внутри
import pandas as pd
import numpy as np
from IPython.display import display

np.random.seed(42)  # фиксируем генератор случайных чисел

data = pd.read_csv('penguins_binary_classification.csv')
print('Размер датасета:', data.shape)
display(data.head())


Размер датасета: (274, 7)


,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,2007
3,Adelie,Torgersen,36.7,19.3,193.0,3450.0,2007
4,Adelie,Torgersen,39.3,20.6,190.0,3650.0,2007


In [3]:

# Вспомогательные функции для разбиения, стандартизации и метрики f1

def simple_train_test_split(X: pd.DataFrame, y: pd.Series, test_size: float = 0.2, random_state: int = 42):
    """Простое разбиение без sklearn, чтобы всё было прозрачно."""
    np.random.seed(random_state)
    indices = np.arange(len(y))
    np.random.shuffle(indices)
    split_point = int(len(y) * (1 - test_size))
    train_idx = indices[:split_point]
    test_idx = indices[split_point:]
    return (
        X.iloc[train_idx].reset_index(drop=True),
        X.iloc[test_idx].reset_index(drop=True),
        y.iloc[train_idx].reset_index(drop=True),
        y.iloc[test_idx].reset_index(drop=True),
    )


def standardize_with_train(X_train: pd.DataFrame, X_test: pd.DataFrame):
    """Масштабируем признаки по среднему и СКО из train, чтобы признаки были сопоставимы."""
    means = X_train.mean()
    stds = X_train.std(ddof=0)
    stds_replaced = stds.replace(0, 1)  # защита от деления на ноль
    X_train_scaled = (X_train - means) / stds_replaced
    X_test_scaled = (X_test - means) / stds_replaced
    return X_train_scaled, X_test_scaled


def f1_score_manual(y_true: np.array, y_pred: np.array) -> float:
    """Ручной расчёт f1-score без внешних библиотек."""
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    precision = tp / (tp + fp + 1e-12)
    recall = tp / (tp + fn + 1e-12)
    if precision + recall == 0:
        return 0.0
    return 2 * precision * recall / (precision + recall)


def run_experiments(X_train, y_train, X_test, y_test, title: str):
    """Перебираем все комбинации регуляризации и оптимизации, выводим f1-score."""
    print(title)
    print('-' * len(title))
    results = []
    reg_options = [
        ("none", 0.0),
        ("l1", 0.01),
        ("l2", 0.01),
        ("l1l2", 0.01),
    ]
    solvers = ["gd", "sgd", "newton"]

    for reg_type, strength in reg_options:
        for solver in solvers:
            model = MyBinaryLogisticRegression(
                reg_type=reg_type,
                reg_strength=strength,
                l1_ratio=0.5,
                lr=0.1,
                max_iter=3000,
                tol=1e-5,
                batch_size=16,
                random_state=42,
            )
            model.fit(X_train, y_train, method=solver)
            preds = model.predict(X_test)
            f1 = f1_score_manual(y_test.values, preds)
            results.append(
                {
                    "Регуляризация": reg_type,
                    "Оптимизация": solver,
                    "F1": round(float(f1), 4),
                }
            )

    results_df = pd.DataFrame(results)
    display(results_df.sort_values(by="F1", ascending=False).reset_index(drop=True))


In [4]:

# Вариант 1: one-hot кодирование островов
# Целевая переменная: 1 - Gentoo, 0 - Adelie

target = (data['species'] == 'Gentoo').astype(int)
features_one_hot = pd.get_dummies(
    data.drop(columns=['species']),
    columns=['island'],
    drop_first=False
)

X_train_oh, X_test_oh, y_train, y_test = simple_train_test_split(
    features_one_hot, target, test_size=0.2, random_state=42
)
X_train_oh, X_test_oh = standardize_with_train(X_train_oh, X_test_oh)

run_experiments(X_train_oh, y_train, X_test_oh, y_test, "One-hot кодирование островов")


One-hot кодирование островов
----------------------------


C:\Users\dermi\AppData\Roaming\Python\Python313\site-packages\numpy\linalg\_linalg.py:2792: RuntimeWarning: overflow encountered in dot
  sqnorm = x.dot(x)


,Регуляризация,Оптимизация,F1
0,none,gd,1.0000
1,none,sgd,1.0000
2,l1,gd,1.0000
3,l1,sgd,1.0000
4,l1l2,sgd,1.0000
5,l2,gd,1.0000
6,l2,sgd,1.0000
7,l2,newton,1.0000
8,l1l2,newton,1.0000
9,l1l2,gd,1.0000


In [5]:

# Вариант 2: целевое кодирование (target encoding) островов

target = (data['species'] == 'Gentoo').astype(int)

data_te = data.copy()
data_te['target'] = target
# Для target encoding считаем средний ответ по каждому острову
island_target_mean = data_te.groupby('island')['target'].mean()
data_te['island_te'] = data_te['island'].map(island_target_mean)

# Оставляем числовые признаки + закодированный остров
X_te = data_te[
    [
        'bill_length_mm',
        'bill_depth_mm',
        'flipper_length_mm',
        'body_mass_g',
        'year',
        'island_te',
    ]
]

X_train_te, X_test_te, y_train_te, y_test_te = simple_train_test_split(
    X_te, target, test_size=0.2, random_state=42
)
X_train_te, X_test_te = standardize_with_train(X_train_te, X_test_te)

run_experiments(X_train_te, y_train_te, X_test_te, y_test_te, "Target encoding островов")


Target encoding островов
------------------------


,Регуляризация,Оптимизация,F1
0,none,gd,1.0000
1,none,sgd,1.0000
2,l1,gd,1.0000
3,l1,sgd,1.0000
4,l2,newton,1.0000
5,l1,newton,1.0000
6,l2,gd,1.0000
7,l2,sgd,1.0000
8,l1l2,sgd,1.0000
9,l1l2,gd,1.0000


# Теоретическая часть

Пусть данные имеют вид
$$
(x_i, y_i), \quad y_i \in \{1, \ldots,M\}, \quad i \in \{1, \ldots, N\},
$$
причем первая координата набора признаков каждого объекта равна $1$.
Используя `softmax`-подход, дискриминативная модель имеет следующий вид
$$
\mathbb P(C_k|x) = \frac{\exp(\omega_k^Tx)}{\sum_i \exp(\omega_i^Tx)}.
$$
Для написания правдоподобия удобно провести `one-hot` кодирование меток класса, сопоставив каждому объекту $x_i$ вектор $\widehat y_i = (y_{11}, \ldots, y_{1M})$ длины $M$, состоящий из нулей и ровно одной единицы ($y_{iy_i} = 1$), отвечающей соответствующему классу. В этом случае правдоподобие имеет вид
$$
\mathbb P(D|\omega) = \prod_{i = 1}^{N}\prod_{j = 1}^M \mathbb P(C_j|x_i)^{y_{ij}}.
$$
Ваша задача: вывести функцию потерь, градиент и гессиан для многоклассовой логистической регрессии. Реализовать матрично. На синтетическом примере продемонстрировать работу алгоритма, построить гиперплоскости, объяснить классификацию